# 🔬 Segmentation panoptique des noyaux et des tissus dans le mélanome avancé 🧬 

## Alix ANNERAUD et Quentin BERAL

Quentin : 

Bonjour à tous,

Nous allons vous présenter notre projet de deep learning sur la segmentation et classification des noyaux cellulaires dans le mélanome.

Dans ce projet, nous avons travaillé sur l'application de méthodes et modèles de deep  sur la segmentation et classification des noyaux cellulaires et tissus dans les images histopathologiques du mélanome en utilisant des techniques de deep learning.

Ce projet se base sur le concours PUMA organisé par le consortium de recherche sur le mélanome avancé, qui vise à améliorer les biomarqueurs pronostiques pour le mélanome avancé.

Nous nous basons sur l'article scientifique : "PUMA: A dataset for deep learning-based nuclei and tissue segmentation in advanced melanoma with improved biomarker potential" publié dans GigaScience en 2024.

Ce travail s'inscrit dans une démarche d'amélioration des biomarqueurs pronostiques pour le mélanome avancé.

# 1. 📜 Contexte

## 🩺 Le mélanome et ses biomarqueurs

- 🏆 Compétition : PUMA
- 🦠 Mélanome : cancer agressif de la peau en augmentation
- ❌ 50% des patients ne répondent pas à l'immunothérapie
- Les lymphocytes infiltrant la tumeur (TILs) = biomarqueur pronostique clé
- ❌ Limites actuelles : évaluation manuelle subjective et variable

Alix :

Tout d'abord, pourquoi ce sujet précis ? Eh bien, nous avons passé beaucoup de temps à trouver un sujet qui nous intéresse et qui est pertinent pour la recherche actuelle. En effet, beaucoup de sujets sont déjà traités, vu et revu il est donc difficile d'être innovant sur des sujets un minimum "commun". Ainsi, ce sujet était parfait car il est niche, ce qui nous laisse une grande liberté d'exploration, et il est aussi très intéressant car il touche à la lutte contre le cancer, un sujet qui est très important pour nous tous.

Le mélanome est une forme particulièrement agressive de cancer cutané dont l'incidence augmente.

Bien que le mélanome primaire soit souvent traité par excision chirurgicale, le mélanome métastatique nécessite une immunothérapie.

Malheureusement, environ la moitié des patients ne répondent pas à cette thérapie, qui est coûteuse et potentiellement toxique.

Il a été identifié qu'il existe des biomarqueurs pronostiques, tels que les lymphocytes infiltrant la tumeur (TILs), qui peuvent aider à prédire la réponse au traitement. Mais leur évaluation manuelle est subjective, chronophage et souffre d'une variabilité inter-observateurs considérable.

Ces limitations motivent le développement d'approches automatisées basées sur du machine/deep learning pour quantifier et caractériser les TILs de manière plus objective et reproductible.

## 🤔 Défis spécifiques au mélanome

- 🧫 Cellules de mélanome mimétiques imitant d'autres types cellulaires
- 🤖 Modèles génériques (HoverNet sur PanNuke) : performance sous-optimale
- ❌ Confusion fréquente entre cellules tumorales/lymphocytes/stromales

→ Nécessité d'un modèle spécifique au mélanome

Quentin :

L'analyse histopathologique du mélanome présente des défis spécifiques qui compliquent l'application des modèles d'IA existants :

1. Les cellules de mélanome ont une capacité unique à imiter morphologiquement d'autres types cellulaires, rendant leur identification automatique particulièrement difficile.

2. Les modèles génériques comme HoverNet, même lorsqu'ils sont pré-entraînés sur des datasets comme PanNuke (qui inclut des échantillons de peau), montrent des performances sous-optimales sur les images de mélanome.

3. Ces modèles confondent fréquemment les cellules tumorales avec des lymphocytes ou des cellules stromales, compromettant ainsi l'évaluation précise des TILs.

4. Les méthodes manuelles actuelles souffrent également d'une grande variabilité dans l'évaluation de la localisation des TILs (intratumorale vs péritumorale), qui est pourtant cruciale pour évaluer leur valeur pronostique.

Ces défis justifient le développement d'un modèle spécifiquement adapté au mélanome, entraîné sur des données annotées de haute qualité.

## 📊 Jeu de données (PUMA)

- Images de **microscope** x40 (1024×1024 pixels) de **régions tumorale** :
  - 155 mélanomes primaires
  - 155 mélanomes métastatiques
- Images contextuelles de **microscope** x40 (5120x5120 pixels)
- **Annotations** de zones en GeoJSON validées par un dermato-pathologiste :
  - Noyaux :
    - Zone : polygone
    - Classe : tumeur, lymphocytes, plasmocytes ... (10 classes)
  - Tissus : tumeur, stroma, épiderme, vaisseaux sanguins ... (5 classes)
    - Zone : polygone(s)
    - Classe : tumeur, épiderme, vaisseaux sanguins ... (5 classes)

Alix :

Le jeu de données : 1ere BD spécifique au mélanome pour la segmentation panoptique des noyaux cellulaires et des tissus :

- Il comprend 155 régions d'intérêt de mélanomes primaires et 155 de mélanomes métastatiques, offrant ainsi un échantillonnage représentatif de la diversité morphologique de cette pathologie.

- Les images sont des coupes/lames colorées numérisées à un grossissement de 40x avec une résolution de 1024×1024 pixels.

- Chaque ROI est accompagnée d'une ROI contextuelle de 5120×5120 pixels centrée autour d'elle, permettant d'analyser l'environnement tissulaire plus large.

- Les annotations des noyaux cellulaires identifient 10 types distincts : tumeur, lymphocytes, plasmocytes, histiocytes, mélanophages, neutrophiles, cellules stromales, épithélium, endothélium et cellules apoptotiques.

- Les annotations tissulaires comprennent 5 catégories : tumeur, stroma, épithélium, vaisseaux sanguins et zones nécrotiques.

- Toutes les annotations ont été réalisées par un professionnel médical et vérifiées par un dermato-pathologiste certifié, garantissant leur qualité et leur pertinence clinique.

Ce jeu de données unique permet de développer et d'évaluer des modèles spécifiques au mélanome avec un niveau de granularité sans précédent.

## 🖼️ Jeu de données (PUMA)

![./images/d4b50cb2-a768-446b-abd3-049f6438d1c0.png](./images/d4b50cb2-a768-446b-abd3-049f6438d1c0.png)

Alix : 

Cette image représente un exemple d'une région d'intérêt (ROI) du jeu de données PUMA avec ses annotations. On peut y observer une coupe, avec en superposition les annotations manuelles validées.

Sur cette image, nous pouvons distinguer différentes régions tissulaires identifiées par des contours colorés, ainsi que les noyaux cellulaires individuels annotés. Les noyaux sont classifiés selon leur type cellulaire (tumeur, lymphocytes, etc.), chacun étant représenté par une couleur spécifique.

Cette annotation précise et détaillée permet d'entraîner des modèles de segmentation panoptique capables à la fois d'identifier les régions tissulaires et de détecter/classifier les noyaux cellulaires individuels, ce qui est essentiel pour l'analyse quantitative des TILs et d'autres biomarqueurs cellulaires dans le mélanome.

## 🖼️ Jeu de données (PUMA)

![./images/6f3736bd-a0f5-42a7-8d54-98852b7e6af1.png](./images/6f3736bd-a0f5-42a7-8d54-98852b7e6af1.png)

Alix : 
    
1. Classes de noyaux n'est pas équilibrée => il y a beaucoup plus de noyaux tumoraux que de lymphocytes, plasmocytes, etc. Cela est du au fait que nous nous concentrons sur des exemples de mélanome, qui sont des tumeurs. 

2. De plus, la représentativité des classes de noyaux sont du à des facteurs biologiques, par exemple, les cellules apoptotiques sont très rares, car elles sont en train de mourir (comme une étoile filancte), ce qui explique leur faible nombre dans le jeu de données.

## 🖼️ Jeu de données (PUMA)

![./images/6f3736bd-a0f5-42a7-8d54-98852b7e6af1.png](./images/34b70ca9-06c8-4241-b2e6-5be0dc1ddc50.png)

Alix : 
    
Idem pour les classes de tissus, il y a un déséquilibre entre les classes. Par exemple, il y a beaucoup plus de tissus tumoraux que de tissus épithéliaux ou vasculaires. 

Ainsi, ce déséquilibre entre les classes de noyaux et de tissus peut poser des problèmes lors de l'entraînement des modèles de deep learning, car ils peuvent avoir tendance à privilégier les classes majoritaires au détriment des classes minoritaires. Cela peut entraîner une mauvaise performance du modèle sur les classes moins représentées, ce qui est problématique pour une application clinique où chaque type cellulaire est important pour le diagnostic et le pronostic.

## 🎯 Principe

Il y a 2 tâches : 

1. Noyaux cellulaires
   1. Segmentation
   2. Classification (10 classes)
2. Segmentation des tissus
   1. Segmentation
   2. Classification (5 classes)

Quentin : 

Pour permettre le diagnostic et le pronostic du mélanome, il est essentiel de segmenter et classifier les noyaux cellulaires et les tissus dans les images histopathologiques.

Il s'agit de deux tâches de segmentation panoptique, où chaque pixel d'une image est classé en fonction de son appartenance à une catégorie spécifique.

Attention, il faut bien faire la différence entre la segmentation panoptique et la segmentation sémantique. 

La segmentation panoptique consiste à segmenter chaque pixel d'une image en lui attribuant une étiquette de classe, tout en tenant compte de la hiérarchie des objets présents dans l'image. En revanche, la segmentation sémantique se concentre uniquement sur l'attribution d'étiquettes de classe aux pixels, sans tenir compte de la hiérarchie ou des relations entre les objets.

Ainsi, il à la fois une tâche de segmentation sémantique (identifier les classes) et une tâche de détection d'objets (identifier les instances).

Il faut donc 2 modèles : un pour la segmentation des noyaux et un pour la segmentation des tissus.

# 2. 🛠️ Méthodologie

1. État de l'art 
2. Prétraitement
3. Entraînement
4. Évaluation
5. Interprétation

Alix : 

La méthodologie de ce projet se décline en plusieurs étapes clés :

1. **État de l'art** : Analyse des méthodes existantes pour la segmentation et la classification des noyaux cellulaires et des tissus dans les images histopathologiques, en mettant l'accent sur les défis spécifiques au mélanome. Cette étape est réalisée en consultant la littérature scientifique et également en demandant à ChatGPT de faire une revue de la littérature sur le sujet.
2. **Prétraitement** : Préparation des données, y compris la normalisation des images, l'augmentation des données et la création de masques d'annotation pour les noyaux et les tissus.
3. **Entraînement** : Entraînement du modèle sur le jeu de données PUMA, en utilisant des techniques d'optimisation et de régularisation pour améliorer la performance.
4. **Évaluation** : Évaluation des performances du modèle sur un ensemble de test indépendant, en utilisant des métriques appropriées pour la segmentation et la classification.
5. **Interprétation** : Analyse des résultats obtenus, identification des forces et des faiblesses du modèle, et discussion des implications cliniques potentielles.

# 3. 💻 Projet

Nous avons essayé les modèles suivants :
- Segmentation et classification des noyaux :
  - CellVit++
- Segmentation et classification des tissus :
  - HoVer-Net
  - nnU-Net

Quentin : 

Pour la segmentation et classification des noyaux cellulaires, nous avons opté pour **CellVit++** qui représente l'état de l'art actuel dans ce domaine. Ce modèle utilise une architecture de Vision Transformer spécialisée qui combine efficacement segmentation d'instance et classification dans un seul pipeline.

Pour la segmentation et classification des tissus, nous avons exploré deux approches :

**HoVer-Net** : Un modèle spécifiquement conçu pour la segmentation et classification des noyaux cellulaires dans les images histopathologiques. Son architecture utilise des cartes de gradients horizontaux et verticaux (d'où son nom) pour séparer les noyaux qui se touchent, ce qui est particulièrement utile dans les images denses de mélanome.

**nnU-Net** : Une implémentation automatisée et auto-configurante du U-Net classique. Ce framework détermine automatiquement tous les hyperparamètres (prétraitement, augmentation de données, architecture réseau, post-traitement) en fonction des caractéristiques du jeu de données. Il représente l'état de l'art pour de nombreuses tâches de segmentation médicale.

Notre objectif était de comparer ces approches pour déterminer celle qui offre le meilleur équilibre entre précision et efficacité computationnelle pour l'analyse du mélanome.

## 🌐 HoVer-Net

![](./images/diagram.png)


Alix : 

...

## ✨ CellVit++

![](./images/network_large.png)

Quentin : 

CellVit++ représente une avancée significative dans le domaine de la segmentation et classification des noyaux cellulaires dans les images histopathologiques. Son architecture innovante repose sur plusieurs éléments clés :

1. **Architecture de Vision Transformer** : À la différence des réseaux convolutifs traditionnels, CellVit++ utilise un transformeur de vision qui traite l'image en séquence de patches, permettant de capturer des relations à longue distance entre les éléments de l'image. Cette approche est particulièrement efficace pour les structures cellulaires complexes du mélanome.

2. **Encodage contextuel** : Le modèle génère des embeddings riches pour chaque région d'intérêt, capturant non seulement les caractéristiques locales de chaque cellule mais aussi son contexte environnant, ce qui est crucial pour distinguer certains types cellulaires dans le microenvironnement tumoral.

3. **Pipeline en deux étapes** : 
   - **Segmentation** : Identifie d'abord les contours précis de chaque noyau cellulaire
   - **Classification** : Classifie ensuite chaque noyau détecté selon les 10 types cellulaires du dataset PUMA

4. **Transfert d'apprentissage efficace** : Le modèle est pré-entraîné sur de larges datasets histopathologiques génériques, puis affiné spécifiquement sur le mélanome, ce qui permet d'obtenir d'excellentes performances même avec un nombre limité d'échantillons annotés.

Cette architecture moderne permet à CellVit++ d'atteindre des performances à l'état de l'art tout en restant computationnellement plus efficace que des approches concurrentes comme HoVer-Net.

## ✨ CellVit++

Points forts :

- ⚡ Adaptabilité : on ne réentraîne que le classifieur (pas le modèle entier)
- 🚀 Performance à l'état de l'art sur 7 jeux de données, même en **zero-shot**
- ⚖️ Comparable aux méthodes lourdes (HoVer-Net, SoftCTM)
- ✅ Jusqu'à **90 % de temps de calcul en moins**
- ⚡ Classifieur entraîné en **< 2 min**
- 🖥️ Interface web interactive (annotation, AutoML)


Alix :

CellVit++ présente plusieurs avantages notables :

- **Adaptabilité** : Il permet de ne réentraîner que le classifieur, ce qui réduit considérablement le temps et les ressources nécessaires pour l'adapter à de nouveaux jeux de données.
- **Performance à l'état de l'art** : Il a démontré des performances exceptionnelles sur 7 jeux de données différents, même dans des scénarios de "zero-shot", où le modèle n'a pas été spécifiquement entraîné sur ces données.
- **Efficacité computationnelle** : Il est comparable aux méthodes lourdes comme HoVer-Net et SoftCTM, mais avec des temps de calcul réduits de jusqu'à 90 %.
- **Rapidité d'entraînement** : Le classifieur peut être entraîné en moins de 2 minutes, ce qui est un atout majeur pour les chercheurs et cliniciens.
- **Interface web interactive** : Il propose une interface conviviale pour l'annotation et l'AutoML, facilitant son utilisation par des professionnels de la santé et des chercheurs.


### Sweep Run

![./images/sweep.png](./images/sweep.png)

Quentin : 

Avant d'entraîner le modèle final, on a utilisé **Weights & Biases Sweep** pour explorer automatiquement différentes combinaisons d’hyperparamètres. Cette étape est cruciale pour identifier les réglages les plus performants sans avoir à les tester manuellement un par un.

Le sweep a permis de tester plusieurs valeurs pour des paramètres comme le taux d’apprentissage, la taille du batch, les coefficients de régularisation, ou encore les paramètres spécifiques à l’optimiseur. Chaque configuration a été évaluée selon des métriques de validation (AUROC, F1-Score, etc.), et Wandb a automatiquement sélectionné les runs les plus prometteurs.

Grâce à ce processus, j’ai pu **trouver une combinaison d’hyperparamètres optimale**, qui a ensuite été utilisée pour entraîner le modèle final, dont les performances sont présentées dans les graphiques précédents. Cela garantit une meilleure efficacité d’apprentissage et une généralisation plus robuste.

### Hyperparamètres

![./images/resultats_sweep.png](./images/resultats_sweep.png)

Alix : 

Sur cette slide, nous analysons les performances de notre modèle à travers un sweep d’hyperparamètres, avec comme métrique principale l’AUROC sur le jeu de validation.
- **Graphique en haut à gauche** : Évolution de l’AUROC au fil du temps. On observe une progression globale, avec une stabilisation autour de **0.930 à 0.934**, ce qui indique une convergence vers des combinaisons efficaces.
- **Encart à droite** : Importance des hyperparamètres par rapport à l’AUROC. Les plus influents sont :
    - model.hidden_dim (taille des couches cachées),
    - training.optimizer.lr (taux d’apprentissage),
    - training.drop_rate (taux de dropout).
        
On note une **corrélation positive du hidden_dim** et du **dropout**, et une **corrélation négative pour le learning_rate**, indiquant qu’un taux trop élevé dégrade la performance.
        
- **Graphique en bas** : Visualisation parallèle des hyperparamètres. Chaque ligne correspond à un essai, colorée selon la valeur de l’AUROC.
    - Les lignes les plus claires (meilleurs scores) se regroupent autour de certaines plages :
        - hidden_dim autour de 300-450,
        - drop_rate entre 0.2 et 0.4,
        - learning_rate entre 2e-5 et 5e-5.

**Le sweep a permis d’identifier une configuration optimale et de mieux comprendre l’influence de chaque paramètre sur la performance du modèle.


## 🚀 nnU-Net

- Même structure que U-Net classique.
- Ajustements minimes :  
  - **Leaky ReLU** à la place du **ReLU**  
  - Instance Normalization au lieu de BatchNorm  
- Tout se joue dans le **pipeline automatique** :  
  - pré-traitement,  
  - entraînement,  
  - inférence,  
  - post-traitement.

Quentin : 

nnU-Net (no-new U-Net) représente une approche révolutionnaire dans le domaine de la segmentation médicale. Contrairement à ce que son nom pourrait suggérer, ce n'est pas seulement une architecture de réseau, mais un **framework complet d'auto-configuration** pour la segmentation d'images médicales.

L'architecture de base reste similaire au U-Net classique, avec quelques modifications mineures comme l'utilisation de Leaky ReLU à la place de ReLU standard et l'Instance Normalization à la place de la Batch Normalization, qui améliore la stabilité lors de l'entraînement sur de petits batchs.

Ce qui distingue vraiment nnU-Net, c'est son pipeline automatisé qui configure tous les aspects du processus :

1. **Prétraitement intelligeant** : Analyse automatique des caractéristiques du dataset (résolution, intensités, anisotropie) pour déterminer la normalisation optimale et le rééchantillonnage.

2. **Configuration du réseau** : Ajuste automatiquement la profondeur, les tailles d'entrée et les paramètres architecturaux en fonction des dimensions des images et des ressources disponibles.

3. **Stratégie d'entraînement adaptative** : Sélection de la fonction de perte appropriée, adaptation des paramètres d'augmentation de données et régulation des hyperparamètres d'optimisation.

4. **Post-traitement optimisé** : Application de processus de raffinement comme des opérations morphologiques conditionnelles basées sur les statistiques du jeu d'entraînement.

En essence, nnU-Net transforme l'art de la segmentation médicale en une science reproductible et automatisée, éliminant le besoin d'ajustements manuels fastidieux des hyperparamètres.

## 🚀 U-Net

![](./images/68747470733a2f2f692e696d6775722e636f6d2f6a6544567071462e706e67.png)


Alix : 

Je vous explique brièvement comment fonctionne le U-Net.
C’est une architecture en forme de U avec deux chemins :

1. D’abord une phase de contraction, qui capture le contexte global. Elle est constitué de plusieurs blocs de convolution, suivis d'une opération de max pooling pour réduire la taille de l'image tout en augmentant le nombre de canaux. Chaque bloc de convolution est suivi d'une activation ReLU et d'une normalisation par lot (BatchNorm). Cette phase permet au réseau d'apprendre des caractéristiques à différentes échelles.

2. Puis une phase d’expansion, qui restaure la résolution avec des informations plus précises grâce aux connexions skip. Elle utilise des convolutions transposées pour augmenter la taille de l'image, suivies de convolutions classiques. Les connexions skip entre les couches correspondantes de la phase de contraction et d'expansion permettent de conserver les détails spatiaux perdus lors de la contraction.
 
Ces connexions permettent au réseau de localiser avec précision, même sur de petites structures.

À la sortie, une convolution 1x1 donne la carte des classes. 
 
Et pour les grandes images, on fait une prédiction par morceaux avec des marges pour éviter les artefacts de bord.

## 🚀 nnU-Net

Avantages :

✅ nnU-Net = pipeline automatique, pas de nouvelle architecture  
✅ Généralise très bien à des cas variés  
✅ Montre que le **contexte d'entraînement** est souvent plus critique que l’architecture elle-même

🧪 Idéal pour benchmark, pré-entraînement, ou compétition

Quentin : 

L'avantage principal de nnU-Net réside dans son automatisation quasi-totale du pipeline de segmentation médicale. Il s'agit d'une approche "plug-and-play" qui permet aux chercheurs et cliniciens de se concentrer sur l'analyse des résultats plutôt que sur les aspects techniques de configuration du modèle.

L'aspect remarquable de nnU-Net est sa capacité à s'adapter à quasiment n'importe quel type d'image médicale, qu'il s'agisse de scans IRM, CT, échographies ou, comme dans notre cas, d'images histopathologiques. Cette adaptabilité en fait un outil particulièrement puissant pour établir des benchmarks et comparer différentes approches sur une base commune.

Par ailleurs, cette approche démontre que les performances d'un modèle de segmentation dépendent souvent davantage du contexte d'entraînement (prétraitement, augmentation, sélection des données) que de l'architecture elle-même. C'est une leçon importante pour le domaine de l'IA médicale : parfois, optimiser l'existant peut s'avérer plus efficace que créer de nouvelles architectures complexes.

Enfin, nnU-Net est particulièrement adapté aux contextes de compétitions scientifiques comme les challenges MICCAI, où il a remporté de nombreuses victoires. Il constitue également une excellente base pour le pré-entraînement avant un fine-tuning avec des architectures plus spécialisées.

# 4. 📈 Résultats

## ✨ CellVit++

![./images/best_run.png](./images/best_run.png)

Alix : 

Voici l’évolution des principales métriques pour notre modèle CellViT-SAM-H sur 50 étapes d'entraînement. On observe d’abord une baisse régulière de la perte d'entraînement, signe que le modèle apprend efficacement à partir des données.

Du côté de la validation, la perte diminue nettement au début puis se stabilise, avec une légère hausse après l’étape 10, ce qui indique un début de surapprentissage. Malgré cela, les métriques de performance restent très solides : l’AUROC dépasse 0.93, ce qui montre une très bonne capacité à distinguer les classes. Le F1-Score se stabilise autour de 0.767, ce qui traduit un bon équilibre entre précision et rappel. Enfin, la précision moyenne (Average Precision) progresse rapidement avant de se stabiliser autour de 0.445, ce qui reste un très bon score pour ce type de tâche.

En résumé, le modèle montre une excellente capacité d’apprentissage et de généralisation, avec un léger surapprentissage à surveiller. Un arrêt anticipé de l'entraînement autour de l’étape 15 à 20 pourrait permettre d’optimiser encore davantage la performance sur données de validation.

## 🌐 HoVer-Net

![./images/hovernet.png](./images/hovernet.png)

Quentin : 

**🔹 Loss hv mse**
- Baisse rapide au début, stabilisation vers 0.05 dès l’époque 15.
- Bonne convergence de la branche de régression (vecteurs de flux).

**🔹 Loss tp bce**
- Diminution progressive, plateau autour de 0.65.
- Légère remontée à la fin → début possible d’overfitting.

**🔹 Valid np acc**
- Hausse régulière, atteint ~0.885.
- Précision stable dès l’époque 35.

**🔹 Valid np dice**
- Amélioration constante, stabilisation vers 0.89.
- Segmentation précise et cohérente avec la classification.

**Conclusion**
- Modèle bien entraîné, performances stables.
- Léger overfitting à surveiller côté classification.
- Bon équilibre entre segmentation et détection.

## 🚀 nnU-Net

❌ Impossible de lancer un apprentissage : 
  - Environnement incompatible ?
  - Dépendances ?

🧊 Dépannage compliqué car aucun message d'erreur (processus bloqué).

```
Date: Tue Apr 29 12:41:03 PM CEST 2025
Noeud: juliet4
Job ID: 22392
Partition: mesonet
Compte: m25031
GPUs alloués: 0
nnUNet_preprocessed set to: /home/alanneraud/Deep_learning_project/nnunet/nnUNet_preprocessed
nnUNet_raw set to: /home/alanneraud/Deep_learning_project/nnunet/nnUNet_raw
nnUNet_results set to: /home/alanneraud/Deep_learning_project/nnunet/nnUNet_results
Activation de l'environnement uv: /home/alanneraud/Deep_learning_project/nnunet/.venv
Python 3.10.17
PyTorch et nnUNet (avec trainer custom) sont supposés être installés dans l'environnement uv.
Vérification de la disponibilité du GPU via PyTorch...
PyTorch version: 2.7.0+cu126
CUDA available: True
Device count: 1
Current device: 0
Device name: NVIDIA A100-SXM4-80GB
Fin de la vérification PyTorch GPU.
Étape 5: Lancement du pré-entraînement (Task 2 - NSCLC) avec nnUNetTrainerIgnoreIndex et plans nnUNetPlans_pretrain...
```

Alix : 

Quant à nnU-Net, bien qu'il nous à été vendu comme un modèle clé en main.

Il n'a pas été possible de le faire fonctionner sur notre environnement.

Nous avons rencontré des problèmes de compatibilité d'environnement et de dépendances, ce qui a rendu le dépannage compliqué.
En effet, le processus d'entraînement se bloquait sans message d'erreur clair, ce qui a rendu difficile l'identification de la source du problème.

Ce modèle ne pouvais pas tourner sur un GPU consummer car il demande trop de mémoire.

# 5. ✅ Conclusion

## 🤔 Critique

- **Occupation des nœuds :**
  - Queue d'attente importante
  - Plantage partiel de `juliet2` et `juliet4`
- **Qualité de l'implémentation des modèles :**
  - Gestion insuffisante des erreurs.
  - Manque de clarté et de robustesse du code.
- **Stabilité et dépendances des modèles :**
  - Difficultés liées à la gestion des versions des librairies.
  - Instabilité potentielle des environnements d'exécution.
- **Maintenance et support des modèles :**
  - Peu de suivi des problèmes signalés (issues).
  - Manque de réactivité sur les contributions (pull requests).

Quentin :

Au cours de ce projet, l'exploitation des modèles existants a présenté plusieurs défis.

1. la gestion des ressources informatiques a été problématique. La queue d'attente pour accéder aux nœuds de calcul était souvent longue, ce qui a entraîné des retards dans le démarrage des expériences. De plus, le nœud `juliet2` a rencontré des problèmes de stabilité, il était impossible d'utiliser les ressources graphiques sur ce dernier, tandis que le nœud `juliet4` a été partiellement bloqué par une tâche qui ne se termine jamais.

2. La qualité de l'implémentation de certains outils s'est avérée perfectible. Par exemple, une gestion limitée des exceptions a entraîné des arrêts inattendus du programme, parfois sans message d'erreur explicite, compliquant le débogage.

3. La stabilité des modèles a été affectée par une gestion imprécise des dépendances. Il a fallu tester itérativement différentes versions de librairies pour assurer la compatibilité et le bon fonctionnement, ce qui a ralenti le processus de développement.

4. Le manque de maintenance active et de support communautaire pour certains des outils utilisés a été un frein. Les problèmes signalés par la communauté (issues GitHub) restent souvent sans réponse, et les propositions d'amélioration (pull requests) ne sont pas intégrées, ce qui soulève des questions sur la pérennité et la fiabilité à long terme de ces outils.

## 🌍 Impact

- **Environnemental :**
  - Fabrication des puces
  - Consommation énergétique

- **Politique :** 
  - Dépendance à l'égard de Taïwan (TSMC) pour la fabrication des puces
  - Dépendance à l'égard de la Chine pour les terres rares
  - Dépendance à l'égard des États-Unis pour les logiciels et les licences (ROCm et CUDA)
  - Indépendance sur la fabrication des machines de lithographie 

- **Sociétal :**
  - Amélioration du diagnostic/pronostic
  - Optimisation du flux de travail
  - Aide à la décision thérapeutique
  - Potentiel de réduction des coûts

Alix : 

Enfin, abordons l'impact de ce type de projet et des technologies sous-jacentes.

Sur le plan **environnemental**, 
1. il faut considérer l'empreinte liée à la fabrication des puces électroniques. Cela inclut l'extraction et la purification de grandes quantités de silicium, issu du sable, pour créer les *wafers* sur lesquels sont gravées les puces mémoire (VRAM) et les processeurs graphiques (GPU). La fabrication des GPU est particulièrement gourmande : leurs *dies* (les puces individuelles) sont très larges, ce qui augmente le taux de perte lors de la découpe des *wafers*.
2. À cela s'ajoute la consommation énergétique significative des usines de fabrication (*fabs*), des centres de données et des phases d'entraînement des modèles.

D'un point de vue **politique et économique**, observons une forte dépendance géostratégique.
1. La production des puces de pointe est concentrée à Taïwan avec TSMC.
2. L'approvisionnement en terres rares, essentielles, dépend largement de la Chine.
3. Les logiciels et écosystèmes comme CUDA ou ROCm sont majoritairement américains.
4. Cependant, l'Europe détient un avantage stratégique majeur avec l'entreprise néerlandaise ASML, qui produit les machines de lithographie EUV (Extreme Ultraviolet) indispensables à la gravure des semi-conducteurs les plus avancés et utilisées par tous les grands fondeurs mondiaux.

Cependant, l'impact le plus direct et positif se situe au niveau **clinique et sociétal**.
1. Ces outils ont le potentiel d'améliorer significativement la précision et la reproductibilité du diagnostic et du pronostic du mélanome.
2. Ils peuvent optimiser le flux de travail des pathologistes, leur permettant de se concentrer sur les cas les plus complexes.
3. En fournissant une analyse plus fine, ils constituent une aide précieuse à la décision thérapeutique, notamment pour prédire la réponse à l'immunothérapie.
4. À terme, cela pourrait même contribuer à une réduction des coûts de santé en évitant des traitements inefficaces.